### Imports

In [1]:
from ultralytics import YOLO
import cv2 as cv
import numpy as np

import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

### Loading Models

In [2]:
#model = YOLO("yolov8n.yaml") # blank with pretrained weights
model = YOLO("D:/Uni Stuff/IP/Coding/Repository/Blimp-Vision-System/runs/detect/5 64/weights/best.pt")  # pretrained model

### Training Models

In [ ]:
#model.train(data="blimps.yaml", epochs=5, batch=64, imgsz=640, name="5 64", exist_ok=True, verbose=True)

### Predicting

In [ ]:
#val_results = model.val(data="testing.yaml", imgsz=640, batch=64, max_det=1, save_hybrid=True)
val_results = model.val(max_det=1)

In [3]:
results = model.predict(source="D:/Uni Stuff/IP/Data/YOLO Data/test/images block 1", save_conf=True, line_thickness=1, max_det=1)



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1314 D:\Uni Stuff\IP\Data\YOLO Data\test\images block 1\image_12750.png: 384x640 1 blimp, 97.2ms
image 2/1314 D:\Uni Stuff\IP\Data\YOLO Data\test\images block 1\image_12751.png: 384x640 1 blimp, 60.4ms
image 3/1314 D:\Uni Stuff\IP\Data\YOLO Data\test\images block 1\image_12752.png: 384x640 1 blimp, 55.4ms
image 4/1314 D:\Uni Stuff\IP\Data\YOLO Data\test\images block 1\image_12753.png: 384x640 1 blimp, 98.5ms
image 5/1314 D:\Uni Stuff\IP\Data\YOLO Da

In [4]:
speed = np.array([])
size = np.array([])
conf = np.array([])

for result in results:
    if len(result.boxes) == 0:
        continue

    speed = np.append(speed, [result.speed['inference']])

    bbox_dim = result.boxes.xywhn
    size = np.append(size, [bbox_dim[0][2] * 1920 * bbox_dim[0][3] * 1080])
    
    conf = np.append(conf, [result.boxes.conf.numpy()[0]])

In [5]:
del results

In [8]:
np.min(conf)

0.2546381652355194

In [71]:
#xlim = [0, 0.05]
#ylim = [0, 172]

plt.hist2d((size / 1000), speed, bins=50, cmap=plt.cm.Blues, norm="log", range=[[0, 20], [55, 70]])

plt.xlabel("Bounding box size ($10^{3} \ pixels^{2}$)")
plt.ylabel("Inference speed ($ms$)")
#plt.xlim(xlim)
#plt.ylim(ylim)
plt.colorbar()
plt.show()

In [73]:
#xlim = [0, 1.0]
#ylim = [90, 130]

plt.hist2d(conf, speed, bins=50, cmap=plt.get_cmap('Blues'), norm="log", range=[[0.2, 1.0], [55, 70]])

plt.xlabel("Confidence")
plt.ylabel("Inference speed ($ms$)")
#plt.xlim(xlim)
#plt.ylim(ylim)
plt.colorbar()
plt.show()

In [65]:
plt.hist2d((size / 1000), conf, bins=50, cmap=plt.get_cmap('Blues'), norm="log", range=[[0, 20], [0.2, 1.0]])

plt.xlabel("Bounding box size ($10^{3} \ pixels^{2}$)")
plt.ylabel("Confidence")
plt.colorbar()
plt.show()

In [56]:
bin_width = 2
min_bin = int(np.min(speed) - (np.min(speed) % bin_width))
max_bin = int(np.max(speed) + bin_width - (np.max(speed) % bin_width))

plt.hist(speed, bins=np.arange(min_bin, max_bin + bin_width, bin_width))

xlim = [int(min_bin - (min_bin % 10)), int(max_bin + 10 - (max_bin % 10))]

plt.xlabel("Inference speed ($ms$)")
plt.ylabel("Frequency")
plt.xlim(xlim)
plt.show()

In [59]:
bin_width = 0.02
min_bin = int(np.min(conf) - (np.min(conf) % bin_width))
max_bin = int(np.max(conf) + bin_width - (np.max(conf) % bin_width))

plt.hist(conf, bins=np.arange(min_bin, max_bin + bin_width, bin_width))

xlim = [0, 1]

plt.xlabel("Confidence")
plt.ylabel("Frequency")
plt.xlim(xlim)
plt.show()